# Experiment with Azure Digital Tween Service
[Location](https://explorer.digitaltwins.azure.net/?tid=76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61&eid=digitaltwins.api.weu.digitaltwins.azure.net)

## Set up

### Create Service Instance
Create instance as described [here](https://learn.microsoft.com/en-gb/azure/digital-twins/quickstart-azure-digital-twins-explorer)

### Define models
To initial define set of models (ontology) was used Office 365 copilot and [prompt](prompts/Define%20models.txt). Some extra tuning was required.
Models are stored in `ontology` folder.
Then models were uploaded to service.

### Ontology
The service define several models as follow:

- Model name: `CentralUnit` - which is main unit of the system exists in one instance. 
    If user want to create new CentralUnit, you must refuse it with message: "Central Unit has to be created manually!"

- Model name: `ControlHub` - which is connection unit between _Central Unit_ and _Termo Sensors_. 

- Model name: `TermoSensor` - which conects to _Control Hub_ and reports temperature value.

## Gen AI
Utilise GenAI to work with toplogy
Add, update, remove digital twin to the graph.

### Init

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
digital_twin_endpoint = os.getenv("AZURE_DIGITAL_TWINS_ENDPOINT")

### Define Tools

Tools that helps LLM to perform tasks on digital twins.

- `get_module_definition` - defines modules and rules of handling digital twins. It's implementation of RAG architecture where user's input _augmented_ with details of the model that needs to be maintained.
- `query_twin` - requests information about _dt_ from Azure Digital Twins Query API.

In [79]:
from azure.digitaltwins.core import DigitalTwinsClient
from azure.identity import DefaultAzureCredential
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.tools import tool
from langchain.tools import HumanInputRun
import json

credential = DefaultAzureCredential()

# Create an instance of the DigitalTwinsClient
client = DigitalTwinsClient(digital_twin_endpoint, credential)

class ModelDefinitionInput(BaseModel):
    model_type: str = Field(description="Model type that definition is required. For example: ControlHub or TermoSensor")

@tool("get-module-definition", args_schema=ModelDefinitionInput)
def get_module_definition(model_type:str) -> str:
    """
        Get the model definition in OTDL format and some usefull instructuion after.
        If model name is not valid, the tool returns NA it means that execution is failed.
    """
    if model_type == "ControlHub":
        with open('ontology/ControlHub.json', 'r') as file:
            definition = file.read()

        instructions = """
        IMPORTANT NOTES:
        1. The model id in $metadata section is dtmi:com:example:ControlHub;1
        2. When requested creation new digital twin it must be performed in 2 steps:
            - create new digital twin of ControlHub model
            - create relationship between new digital twin and CentralUnit twin! ID of CentalUnit twin must be queried through Azure Digital Twins Query API. $relationship Name should be "connects"
        """
        return definition + instructions
    elif model_type == "TermoSensor":
        with open('ontology/TermoSensor.json', 'r') as file:
            definition = file.read()

        instructions = """
        IMPORTANT NOTES:
        1. The model id in $metadata section is dtmi:com:example:TermoSensor;1
        2. When requested creation new digital twin it must be performed in 2 steps:
            - create new digital twin of Termosensor model
            - create relationship between new digital twin and ControlHub twin. 
                $relationship Name should be "connects"
                To make this relationship you must know id of ControlHub twin, this can be obtained by several ways:
                - user can specify id of ControlHub twin in initial request - in this case you have this information.
                - user may specidy name of ControlHub - in this case you must query its id through Azure Digital Twins Query API.
                - if user does not provide neither name nor id you should use available for you human tool and ask the name of ControlHub twin. And then use it to get id as specified above.
        """
        return definition + instructions
    else:
        return "NA"


class UpdateTwinInput(BaseModel):
    twin_id: str = Field(description=
    """ This is mandatoty parameter!
    The unique identifier of the digital twin that is going to be changed or created.
    The identifier passed here must respect name convention for Azure Digital Twins Id
    (alphabetical, numbers, underscore, up to 64 symbols)
    """)
    twin_data: str = Field(description=
    """ This is mandatoty parameter!
    The format has to be Dictionary[string, object] represented as type string
    The dictionary represent digital tween to be sent to the topology.
    The format should be in Azure Digital Twins Definition Language (DTDL).
    
    IMPORTANT! The metadata section should contain id of the model, for example if
    model id is dtmi:com:example:ControlHub;1 metadata section should looks like:
    "$metadata": {
        "$model": "dtmi:com:example:ControlHub;1"
    },

    The format must be valid to be sent as twin_data in following below example.
        
    from azure.digitaltwins.core import DigitalTwinsClient
    
    client = DigitalTwinsClient(digital_twin_endpoint, credential)
    client.upsert_digital_twin(twin_id, twin_data)
    """)

@tool("update-twin-tool", args_schema=UpdateTwinInput)
def update_twin(twin_id, twin_data)->str:
    """
        Adds or Updates the digital twin of Azure Digital Twins Service.
        This function should be used when you want to:
            - create new digital twin
            - update parameters for existed digital twin.

        return: SUCCESS if the operation was successful, otherwise FAIL with description of the error, you're welcome to analyse error and fix it.
    """
    
    # Send data to the topology
    try:
        twin_data_dict = json.loads(twin_data)
        client.upsert_digital_twin(twin_id, twin_data_dict)
        return "SUCCESS\n"
    except Exception as e:
        print("Error sending data to Azure Digital Twins service topology:", str(e))
        return f'FAIL: + {str(e)}\n'
    

class DeleteTwinInput(BaseModel):
    twin_id: str = Field(description=
    """ This is mandatoty parameter!
    The unique identifier of the digital twin that is going to be deleted.
    The identifier is a value of $dtId requested from Azure Digital Twins Query API
    """)

@tool("delete-twin-tool", args_schema=DeleteTwinInput)
def delete_twin(twin_id)->str:
    """
        Delete the digital twin of Azure Digital Twins Service.
        This function should be used when you want to delete digital twin.

        return: SUCCESS if the operation was successful, otherwise FAIL with description of the error, you're welcome to analyse error and fix it.
    """
    
    # Send data to the topology
    try:
        client.delete_digital_twin(twin_id)
        return "SUCCESS\n"
    except Exception as e:
        print("Error deleting digital twin in Azure Digital Twins service topology:", str(e))
        return f'FAIL: + {str(e)}\n'

class UpdateRelationshipInput(BaseModel):
    twin_id: str = Field(description=
    """ This is mandatoty parameter!
    The unique identifier of the digital twin that is going to be the source of the connection.
    The identifier is a value of $dtId requested from Azure Digital Twins Query API
    """)
    relation_id: str = Field(description=
    """ This is mandatoty parameter!
    The unique identifier of the relationship may be provided by user. If not should be
    generated by you. For example can be "connection_1"
    """)
    relation_data: str = Field(description=
    """ This is mandatoty parameter!
    The format has to be Dictionary[string, object] represented as type string
    The dictionary represent relationship to be sent to the topology.
    The format should be in Azure Digital Twins Definition Language (DTDL).
    
    The format must be valid to be sent as relation_data in following below example.
        
    from azure.digitaltwins.core import DigitalTwinsClient
    
    client = DigitalTwinsClient(digital_twin_endpoint, credential)
    client.upsert_relationship(twin_id, relation_id, relation_data)

    It must contain:
    {
        "$relationshipId": "relation_id",
        "$sourceId": "source_id",
        "$relationshipName": "connects",
        "$targetId": "target_id",
        "$metadata": {
            "$model": "model_id"
        }
    }

    where:
     - relation_id - the same unique identifier of the relationship as provided in relation_id argument
     - source_id  - the same unique identifier of the digital twin as provided in twin_id argument
     - "connects" - fixed keyword describes relationship
     - target_id - the unique identifier of the digital twin that is originally was created. This is id of twin that user ask to create or modify.
     - model_id - the unique identifier of the model that is used as source of relationship. Is obtained from model definition for source.
    """)

@tool("update-relationship-tool", args_schema=UpdateRelationshipInput)
def update_relationship(twin_id, relation_id, relation_data)->str:
    """
        Update the relationship between twins in Azure Digital Twins Service.
        This function should be used when you want to:
            - create new relationship
            - update parameters for existed relationship.
    """
    
    # Send data to the topology
    try:
        relation_data_dict = json.loads(relation_data)
        client.upsert_relationship(twin_id, relation_id, relation_data_dict)
        return "SUCCESS\n"
    except Exception as e:
        print("Error updating relationship in Azure Digital Twins service:", str(e))
        return "FAIL\n"

class DeleteRelationshipInput(BaseModel):
    twin_id: str = Field(description=
    """ This is mandatoty parameter!
    The unique identifier of the digital twin that is going to be the source of the connection.
    The identifier is a value of $dtId requested from Azure Digital Twins Query API
    """)
    relation_id: str = Field(description=
    """ This is mandatoty parameter!
    The unique identifier of the relationship to be deleted.
    The identifier is a value of $relationshipId requested from Azure Digital Twins Query API
    """)

@tool("delete-relationship-tool", args_schema=DeleteRelationshipInput)
def delete_relationship(twin_id, relation_id)->str:
    """
        Delete the relationship between twins in Azure Digital Twins Service.
        This function should be used when you want to delete relationship between twins.
    """
    
    # Send data to the topology
    try:
        client.delete_relationship(twin_id, relation_id)
        return "SUCCESS\n"
    except Exception as e:
        print("Error deleting relationship in Azure Digital Twins service topology:", str(e))
        return "FAIL\n"

class QueryTwinInput(BaseModel):
    query: str = Field(description=
    """ This is required parameter!
        Query in Azure Digital Twins Query API format that is going to be executed on Azure Digital Twin Service.
         
        For example: You need to know the details of ControlHub twin.
        From topology you may get model definition for ControlHub iand from definition understand that id is dtmi:com:example:ControlHub;1
        Thaen you can format request as:
        SELECT * FROM digitaltwins T WHERE IS_OF_MODEL(T, 'dtmi:com:example:ControlHub;1')

        The request returns all description of all twins with type ControlHub.
    """)

@tool("query-service", args_schema=QueryTwinInput)
def query_service(query:str) -> str:
    """
        Request information about twins or relationships in Azure Digital Twins Service.
        This function should be used whan you need to get information about particular twin or several items or relationships between them.

        return: response from Azure Digital Twins Query API in JSON format. If function fails it return text started with FAIL word and description of the error, you're welcome to analyse error and fix it.
    """
    if (query == None):
        return "FAIL: Query is not defined!"

    try:
        query_result = client.query_twins(query)
        json_result = json.dumps(list(query_result))
        return json_result
    except Exception as e:
        print("Error deleting digital twin in Azure Digital Twins service topology:", str(e))
        return f'FAIL: + {str(e)}\n'

human_tool = HumanInputRun()

tools = [
    get_module_definition,
    update_twin, delete_twin, query_service,
    update_relationship, delete_relationship,
    human_tool
]

### Configure LLM
Configure Azure Open AI

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import AIMessage
from langchain_core.runnables import Runnable

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"], # 2024-02-15-preview
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYED_NAME"], # gpt-4o
    temperature=0.1
)

### Prompt engineering

In [85]:
from langchain_core.prompts import ChatPromptTemplate

system_message = SystemMessage(
    content="""
    You are helpfull bot that is going to help with creation topology in Azure Digital Twins Service.
    You should not answer with general information, you must help with topology by user request.
    The service define several models as follow:

    - Model name: CentralUnit - which is main unit of the system exists in one instance. 
        You cannot create CentralUnit as it is already defined. If the creation will be requested refuse it.
    - Model name: ControlHub - which is connection unit between CentralUnit and TermoSensors.
    - Model name: `TermoSensor` - which conects to ControlHub and reports temperature value.

    If user want to create, update, delte or request information about any unit you must use appropriate tool(s) and provide valid parameters. 
    Digital Twin Definition Language (DTDL) must be used to communicate with Azyre Digital Twins Service.

    If you requested to create several twins you should think as follows:
        - identify if there is hierarchy between twins, for example is there requested ControlHub and TermoSensors.
        - if hierarchy presents you should start from "higher" twin (ControlHub) and create it accordingly to defined rules, when creation is done continue with
        "lower" twins (TermoSensors) and create them accordingly to defined rules.
        - if hierarchy is not presents you can create twins in any order.
        - once all twins are created you can create relationships between them as requested.

    
    If user want update or delete exsted twin then user must specify twin id. If id is not provided, you must refuse it with message: "DONE with error Twin Id is required!"

    Relationship between digital twins are stored separately from twins. If user request information where connection between twins presens you must request
    relationship information from relationships collection!
    For example for question: what temperature sensor connected to hub with id Node_23 valid query will be:
    SELECT * FROM relationships WHERE $sourceId = 'Node_23'
    Note request was done for relationships not digitaltwins collection.
    
    If you was able to perform requested action, you must provide message: "DONE with success"

    In youe thoughts and communication you never should use markdown or any other non-plain text formatting.
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message,
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

### Execution

Is used ReAct approach that let LLM to think what steps should be done to achieve task and what tool is better fit this needs.
Several agent were cinsidered from [available](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/)

In [80]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#### Control Hub

- Add Control Hub

In [83]:
human_message = HumanMessage(
    content="Please create Control Hub with following parameters: Name: Hub_3, Location: Rivno, Status: on."
)

- Delete Control Hub

In [45]:
human_message = HumanMessage(
    content="Where located Control Hub HUB_4?"
)

#### Add Temperature Sensor

In [43]:
human_message = HumanMessage(
    content="Please create new TermoSensor with following parameters: Name: Sensor_2, Location: Kyiv, Temperature: 22, Status: on. This sensor must be connected to ControlHub id: Hub_8"
)

#### Batch Task

In [90]:
human_message = HumanMessage(
    content="I want to add new location that contains 3 sensors. Status for all of them off. They should be conected to ControlHub that is located in Uszhorod, location of sensor is the same as hub."
)

# Remove all
# human_message = HumanMessage(
#     content="Please remove ControlHub_Odesa and all conected sensors"
# )

# Remove all
human_message = HumanMessage(
    content="Please remove all ControlHubs that are not connected to CentralUnit, also remove all sensor connected to these hubs"
)


#### Start ReAct

In [91]:
# from langchain import hub


result = agent_executor.invoke({"input": human_message.content})




> Entering new AgentExecutor chain...

Invoking: `query-service` with `{'query': "SELECT * FROM digitaltwins T WHERE IS_OF_MODEL(T, 'dtmi:com:example:ControlHub;1')"}`


[{"$dtId": "HUB_4", "$etag": "W/\"997a1df6-b011-4501-8c1d-1ce7bf296081\"", "name": "Hub_4_name", "location": "Kyiv", "status": "on", "$metadata": {"$model": "dtmi:com:example:ControlHub;1", "$lastUpdateTime": "2024-07-05T11:18:21.4425662Z", "name": {"lastUpdateTime": "2024-07-05T11:18:21.4425662Z"}, "location": {"lastUpdateTime": "2024-07-05T11:18:21.4425662Z"}, "status": {"lastUpdateTime": "2024-07-05T11:18:21.4425662Z"}}}, {"$dtId": "Hub_8", "$etag": "W/\"90a716c8-a43a-466c-8a5f-b533c749b6bd\"", "name": "Hub_8", "location": "Kyiv", "status": "on", "$metadata": {"$model": "dtmi:com:example:ControlHub;1", "$lastUpdateTime": "2024-07-05T14:13:43.3039298Z", "name": {"lastUpdateTime": "2024-07-05T14:13:43.3039298Z"}, "location": {"lastUpdateTime": "2024-07-05T14:13:43.3039298Z"}, "status": {"lastUpdateTime": "2024-07-05

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_iJ4yNSkVSbdCubodg2OQoHt8, call_w2JZpIP9sW0czuPZK2KSyspL, call_oK8u5RQ3xHt2AuGu7FMknCEG", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}